In [1]:
import pandas as pd

In [2]:
!ls

a2_m1.json	       a2_m4.json	      model.h5
a2_m1.json.zip	       a2_m4.json.zip	      model.h5.base64
a2_m1.json.zip.base64  a2_m4.json.zip.base64  __pycache__
a2_m2.json	       a_a5wb9u_anything.log  rklib.py
a2_m2.json.zip	       drug_df_clean.csv      scratch_space
a2_m2.json.zip.base64  drug_df.csv	      watsoniotp.broken.pickle
a2_m3.json	       drugsCom_raw.zip       watsoniotp.healthy.pickle
a2_m3.json.zip	       drugsComTest_raw.tsv
a2_m3.json.zip.base64  drugsComTrain_raw.tsv


In [35]:
df= pd.read_csv('drug_df_clean.csv')
df.head()

,Unnamed: 0,Id,review,rating,cleanReview
0,0,163740,"""I&#039;ve tried a few antidepressants over th...",10.0,ive tried antidepressants years citalopram flu...
1,1,206473,"""My son has Crohn&#039;s disease and has done ...",8.0,my son crohns disease done well asacol he comp...
2,2,159672,"""Quick reduction of symptoms""",9.0,quick reduction symptoms
3,3,39293,"""Contrave combines drugs that were used for al...",9.0,contrave combines drugs used alcohol smoking o...
4,4,97768,"""I have been on this birth control for one cyc...",9.0,i birth control one cycle after reading review...


In [36]:
df_drug = df[['Id','rating','cleanReview']].copy()
df_drug.head()

,Id,rating,cleanReview
0,163740,10.0,ive tried antidepressants years citalopram flu...
1,206473,8.0,my son crohns disease done well asacol he comp...
2,159672,9.0,quick reduction symptoms
3,39293,9.0,contrave combines drugs used alcohol smoking o...
4,97768,9.0,i birth control one cycle after reading review...


In [38]:
df_drug.loc[df_drug['rating'] >=8.0,'label'] ='positive' # Label based on rating
df_drug.loc[(df_drug['rating'] >=5.0)&(df_drug['rating']<8.0),'label'] ='neutral'
df_drug.loc[df_drug['rating'] <=4.0,'label'] ='negative'

In [39]:
df_drug.head()

,Id,rating,cleanReview,label
0,163740,10.0,ive tried antidepressants years citalopram flu...,positive
1,206473,8.0,my son crohns disease done well asacol he comp...,positive
2,159672,9.0,quick reduction symptoms,positive
3,39293,9.0,contrave combines drugs used alcohol smoking o...,positive
4,97768,9.0,i birth control one cycle after reading review...,positive


In [40]:
df_drug.to_csv('df_drug.csv')

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer #Vectorize the review column

In [16]:
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
features = tfidf.fit_transform(df_drug.cleanReview.values.astype('U'))
labels   = df_drug.label

In [17]:
features.shape

(215063, 1635780)

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df_drug['cleanReview'],df_drug['label'],random_state=0) #Split into training and test set

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB

In [28]:
#Use cross validation to compare different Machine Learning Algorithm to choose the best performing one
# This will take some time
models = [RandomForestClassifier(n_estimators=200,max_depth=3,random_state=0),LinearSVC(),MultinomialNB(),LogisticRegression(random_state=0,solver='lbfgs',max_iter=2000,multi_class='ovr')]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model,features,labels,scoring='accuracy',cv=CV)
  for fold_idx,accuracy in enumerate(accuracies):
    entries.append((model_name,fold_idx,accuracy))
cv_df = pd.DataFrame(entries,columns=['model_name','fold_idx','accuracy'])

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

LinearSVC gave the highest accuracy. This will be used to build our model

In [31]:
df_drug['features']= features

In [33]:
df_drug.head()

,Id,rating,cleanReview,label,features
0,163740,10.0,ive tried antidepressants years citalopram flu...,positive,"(0, 731165)\t0.0795334950895\n (0, 1475907)..."
1,206473,8.0,my son crohns disease done well asacol he comp...,positive,"(0, 731165)\t0.0795334950895\n (0, 1475907)..."
2,159672,9.0,quick reduction symptoms,positive,"(0, 731165)\t0.0795334950895\n (0, 1475907)..."
3,39293,9.0,contrave combines drugs used alcohol smoking o...,positive,"(0, 731165)\t0.0795334950895\n (0, 1475907)..."
4,97768,9.0,i birth control one cycle after reading review...,positive,"(0, 731165)\t0.0795334950895\n (0, 1475907)..."


In [ ]:
df_drug.shape